In [1]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import cmath
import math
%matplotlib inline
import matplotlib.pyplot as plt
bempp.core.opencl_kernels.set_default_device(0,0)

In [3]:
wavenumber = 4
h = 0.1

In [72]:
grid = bempp.api.shapes.sphere(r=0.2, origin=[0,0,0],h = h)

In [73]:
div_space = bempp.api.function_space(grid, "RWG", 0)
curl_space = bempp.api.function_space(grid, "SNC", 0)

In [74]:
elec = bempp.api.operators.boundary.maxwell.electric_field(div_space, div_space, curl_space, wavenumber)
magn = bempp.api.operators.boundary.maxwell.magnetic_field(div_space, div_space, curl_space, wavenumber)
identity = bempp.api.operators.boundary.sparse.identity(div_space, div_space, curl_space)

In [75]:
@bempp.api.complex_callable
def tangential_trace(x, n, domain_index, result):
    incident_field = np.array([np.exp(1j * wavenumber * x[2]), 0. * x[2], 0. * x[2]])
    result[:] = np.cross(incident_field, n)
trace_fun = bempp.api.GridFunction(div_space, fun=tangential_trace, dual_space=curl_space)

In [76]:
from bempp.api.linalg import lu
#lambda_data = lu(elec, trace_fun)
lambda_data, info_exact = bempp.api.linalg.gmres(elec, trace_fun, tol=1e-5)

In [89]:
elec_pot = bempp.api.operators.potential.maxwell.electric_field(div_space, points, wavenumber)

unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_unit_sphere = unit_sphere.vertices

    
elec_far = bempp.api.operators.far_field.maxwell.electric_field(div_space, vert_unit_sphere, wavenumber)

In [84]:
scattered_field_data = -elec_pot * lambda_data

In [90]:
far_field_data = -elec_far * lambda_data

In [80]:
scattered_field_data

array([[-1.28032069e-02-2.45481533e-03j, -1.29558766e-02-1.22198387e-03j,
        -1.29895207e-02+1.36755448e-05j, ...,
        -4.04726034e-03-2.28483091e-03j, -3.84497589e-03-2.67271022e-03j,
        -3.60397181e-03-3.04338962e-03j],
       [-3.38166933e-06-3.36249692e-06j, -3.69637687e-06-3.04108365e-06j,
        -3.97968482e-06-2.69149771e-06j, ...,
        -3.25991990e-06-2.34898740e-06j, -3.01333285e-06-2.63671499e-06j,
        -2.74075378e-06-2.89986165e-06j],
       [ 1.23421363e-02+3.49401886e-03j,  1.26771211e-02+2.32317939e-03j,
         1.29003697e-02+1.12425001e-03j, ...,
         3.85352325e-03+3.72257977e-03j,  3.49639379e-03+4.05002480e-03j,
         3.10977891e-03+4.34432819e-03j]])

In [88]:
far_field_data_new

array([[ 3.48233595e-13+4.44519421e-13j,  1.08052468e-01+4.42605621e-02j,
         6.00896069e-13+4.60811944e-14j, ...,
         1.30591273e-02+3.77891609e-04j,  8.28824556e-02+1.88948057e-02j,
         1.53221863e-01+3.65525079e-02j],
       [ 2.36700137e-05+1.85479653e-05j, -4.10364445e-06-1.96023108e-06j,
         2.56169141e-05+2.42896069e-05j, ...,
         1.92845286e-03+6.88709234e-04j,  4.27683395e-02+1.55327226e-02j,
        -2.01510157e-03-7.58431710e-04j],
       [ 5.19980910e-02-5.59800964e-03j,  1.12746724e-05+5.38569153e-06j,
        -5.19621691e-02+5.63769045e-03j, ...,
         7.10511743e-02+1.99998552e-03j,  7.31732354e-02+1.04119830e-02j,
         1.15128182e-02+2.67653247e-03j]])

In [91]:
far_field_data

array([[ 3.48233595e-13+4.44519421e-13j,  1.22374972e-01+4.22942867e-02j,
         6.00896069e-13+4.60811944e-14j, ...,
         1.22589187e-02+4.70562827e-04j,  7.45448462e-02+2.00731097e-02j,
         1.53873272e-01+3.64118748e-02j],
       [ 2.36700137e-05+1.85479653e-05j,  4.42496734e-13+3.29652972e-13j,
         2.56169141e-05+2.42896069e-05j, ...,
         9.11279108e-03+3.21277911e-03j,  5.48557202e-02+1.95021863e-02j,
         8.74449883e-04+3.43346864e-04j],
       [ 5.19980910e-02-5.59800964e-03j,  2.19419790e-05+1.31927972e-05j,
        -5.19621691e-02+5.63769045e-03j, ...,
         6.92730550e-02+1.31799875e-03j,  5.77784005e-02+4.24114065e-03j,
         1.16775821e-02+2.74932909e-03j]])

In [107]:
for i in range(far_field_data.shape[1]):
    if abs(far_field_data_new[:,i][0] - far_field_data[:,13][0]) < 0.01 and abs(far_field_data_new[:,i][1] - far_field_data[:,13][1]) < 0.01 and abs(far_field_data_new[:,i][2] - far_field_data[:,13][2]) < 0.01:
        print(far_field_data_new[:,i],i)

[ 0.00867079+0.00452861j -0.03586776-0.01230932j  0.04217872-0.00795834j] 3348
[ 0.02211478+0.00578643j -0.04205495-0.01465078j  0.05656039-0.00197545j] 3583
[ 0.01731588+0.00503691j -0.03995513-0.01386207j  0.053051  -0.00360216j] 3590
[ 0.02122325+0.00648035j -0.04427713-0.01535682j  0.0516227 -0.00368118j] 3624
[ 0.01675506+0.00583563j -0.04216861-0.0145669j   0.04806908-0.00527697j] 3625
[ 0.0207933 +0.00732341j -0.04623233-0.01596531j  0.04667285-0.0053253j ] 3627
[ 0.0161934 +0.00661361j -0.04366088-0.01501886j  0.0432401 -0.00688365j] 3638
[ 0.01323993+0.00442606j -0.03755327-0.01298237j  0.04987141-0.00503695j] 3659
[ 0.02063964+0.00821511j -0.04774434-0.01641948j  0.04195183-0.00684901j] 3669
[ 0.0119694 +0.00602244j -0.04065009-0.01393031j  0.03971327-0.00842227j] 3701
[ 0.01408163+0.00378945j -0.03542441-0.01229567j  0.05450215-0.00344218j] 3847
[ 0.02617146+0.00728121j -0.04608991-0.01605169j  0.05502479-0.00207232j] 3891
[ 0.0254191 +0.00800012j -0.04803377-0.01665597j  0.

In [108]:
far_field_data[:,13]

array([ 0.01790981+0.00618822j, -0.0432381 -0.01493969j,
        0.04804291-0.00517132j])